In [1]:
import pandas as pd
#import pandas_datareader.data as web
print(pd.__version__)

from numpy import nan as NA
import numpy as np

import pprint  

0.25.0


# Data cleaning and preparation

- Significant amount of time (80%) is spent on data prep (loading, cleaning, transform, rearrange)

## Handling missing data

- Missing data occurs commonly
- pandas tries to make working with missing data as painless as possible
    - E.g., descr stats exclude missing data by default

In [2]:
data = pd.Series([1, NA, 3.5, NA, 7])

data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [3]:
data.isnull()

0    False
1     True
2    False
3     True
4    False
dtype: bool

In [4]:
data.notnull()

0     True
1    False
2     True
3    False
4     True
dtype: bool

In [5]:
# Equivalent to: data[data.notnull()]
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

### Filling in missing data

#### pd.obj.fillna

In [6]:
# Create a random data frame with missing data.
df = pd.DataFrame(np.random.randn(7, 3))

df.iloc[:4, 1] = df.iloc[:2, 2] = NA

df

,0,1,2
0,-1.259315,NaN,NaN
1,-0.316003,NaN,NaN
2,-0.135380,NaN,0.351006
3,-1.722350,NaN,-1.639122
4,0.635730,0.506488,-0.362148
5,1.320445,-0.744723,-0.289039
6,-0.274514,-0.936836,-0.960579


In [7]:
df.fillna(0)

,0,1,2
0,-1.259315,0.000000,0.000000
1,-0.316003,0.000000,0.000000
2,-0.135380,0.000000,0.351006
3,-1.722350,0.000000,-1.639122
4,0.635730,0.506488,-0.362148
5,1.320445,-0.744723,-0.289039
6,-0.274514,-0.936836,-0.960579


In [8]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA

df

,0,1,2
0,1.109520,0.685437,1.397496
1,-0.279185,0.358535,1.984311
2,-0.695098,NaN,-0.909163
3,0.057498,NaN,0.414539
4,-0.655092,NaN,NaN
5,-2.357690,NaN,NaN


In [9]:
# Fill nans using forward fill.
df.fillna(method="ffill")

,0,1,2
0,1.109520,0.685437,1.397496
1,-0.279185,0.358535,1.984311
2,-0.695098,0.358535,-0.909163
3,0.057498,0.358535,0.414539
4,-0.655092,0.358535,0.414539
5,-2.357690,0.358535,0.414539


In [10]:
# Use forward fill but with a limit.
df.fillna(method="ffill", limit=2)

,0,1,2
0,1.109520,0.685437,1.397496
1,-0.279185,0.358535,1.984311
2,-0.695098,0.358535,-0.909163
3,0.057498,0.358535,0.414539
4,-0.655092,NaN,0.414539
5,-2.357690,NaN,0.414539


## Data transformation

### Removing dups

#### pd.obj.duplicated()

In [11]:
data = pd.DataFrame({
    'k1': ['one', 'two'] * 3 + ['two'],
    'k2': [1, 1, 2, 3, 3, 4, 4]
})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [12]:
# Return if a row has been observed in a previous row or not.
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

#### pd.drop_duplicates()

In [13]:
# Remove the rows where `duplicated` is True.
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [14]:
# By default the method considers all the columns.
# One can specify a subset of duplicates.

data.duplicated(['k1'])

0    False
1    False
2     True
3     True
4     True
5     True
6     True
dtype: bool

In [15]:
data.duplicated(['k1'], keep="last")

0     True
1     True
2     True
3     True
4    False
5     True
6    False
dtype: bool

In [16]:
print(data)

# By default drop_duplicates keeps the first obeserved value combination
print("\n", data.drop_duplicates(keep="first"))

# keep='last' return the last one
print("\n", data.drop_duplicates(keep="last"))

    k1  k2
0  one   1
1  two   1
2  one   2
3  two   3
4  one   3
5  two   4
6  two   4

     k1  k2
0  one   1
1  two   1
2  one   2
3  two   3
4  one   3
5  two   4

     k1  k2
0  one   1
1  two   1
2  one   2
3  two   3
4  one   3
6  two   4


### Transforming data using function or mapping

In [17]:
data = pd.DataFrame({
    'food': [
        'bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon',
        'pastrami', 'honey ham', 'nova lox'
    ],
    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]
})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [18]:
# Suppose you want to add a column indicating the animal each food came from.

meat_to_animal = {
      'bacon': 'pig',
      'pulled pork': 'pig',
      'pastrami': 'cow',
      'corned beef': 'cow',
      'honey ham': 'pig',
      'nova lox': 'salmon'
}

In [19]:
data['food'].str

In [20]:
# Get the lower case of one column.
# ".str" and ".dt" allow to select a column the algos for a given type.
lowercased = data['food'].str.lower()

print(lowercased)

data['animal'] = lowercased.map(meat_to_animal)

data

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object


,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


#### pd.obj.map()

In [21]:
# Using a single function.

# map() is called on each element of the obj.
# MEM: It's like python map()
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing values
- fillna is a special case of a more general replacement.

#### pd.obj.replace()

In [22]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [23]:
# Replace -999 with nan.
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [24]:
# Replacing multiple values.
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [25]:
# Also columns and index can be replaced using .replace()

### Discretization and binning

#### pd.cut()

In [26]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

bins = [18, 25, 35, 60, 100]

# From "bins" create various categorical variables, then assign
# "ages" to the variables, returning a pd.Categorical array.
cats = pd.cut(ages, bins)

# pd.Categorical.
print(type(cats))
print("cats=\n", cats)

<class 'pandas.core.arrays.categorical.Categorical'>
cats=
 [(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]


In [27]:
# Use int representation of the bins.
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [28]:
# Print the categorical values.
# They are upper close (a, b].
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [29]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [30]:
# Using lower close intervals.
cats = pd.cut(ages, bins, right=False)

cats

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [31]:
# Using custom labels for the categorical variables.
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
cats = pd.cut(ages, bins, labels=group_names)

cats

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [32]:
# Using an integer number of bins will make compute equal-length bins
# between min and max values in the data.

In [33]:
# Uniform distributed numbers between 0 and 1.
np.random.seed(10)
data = np.random.rand(20)

print(data)

[0.77132064 0.02075195 0.63364823 0.74880388 0.49850701 0.22479665
 0.19806286 0.76053071 0.16911084 0.08833981 0.68535982 0.95339335
 0.00394827 0.51219226 0.81262096 0.61252607 0.72175532 0.29187607
 0.91777412 0.71457578]


In [34]:
print(min(data), max(data))

0.003948266327914451 0.9533933461949365


In [35]:
pd.cut(data, bins=4, precision=2)

[(0.72, 0.95], (0.003, 0.24], (0.48, 0.72], (0.72, 0.95], (0.48, 0.72], ..., (0.48, 0.72], (0.72, 0.95], (0.24, 0.48], (0.72, 0.95], (0.48, 0.72]]
Length: 20
Categories (4, interval[float64]): [(0.003, 0.24] < (0.24, 0.48] < (0.48, 0.72] < (0.72, 0.95]]

#### pd.qcut()

In [36]:
# qcut() uses sample quantiles instead of fixed intervals.

data = np.random.randn(1000)

# Use 4 equally space quantiles.
cats = pd.qcut(data, 4)

cats

[(-3.205, -0.64], (-0.0251, 0.602], (0.602, 2.68], (-3.205, -0.64], (-3.205, -0.64], ..., (-3.205, -0.64], (0.602, 2.68], (-3.205, -0.64], (-3.205, -0.64], (-0.0251, 0.602]]
Length: 1000
Categories (4, interval[float64]): [(-3.205, -0.64] < (-0.64, -0.0251] < (-0.0251, 0.602] < (0.602, 2.68]]

In [37]:
# Since the distribution is Gaussian there should be exactly the same proportion
# of numbers in each category.
pd.value_counts(cats)

(0.602, 2.68]       250
(-0.0251, 0.602]    250
(-0.64, -0.0251]    250
(-3.205, -0.64]     250
dtype: int64

In [38]:
# Passing custom quantiles.
cats = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
cats

[(-1.208, -0.0251], (-0.0251, 1.182], (1.182, 2.68], (-1.208, -0.0251], (-3.205, -1.208], ..., (-1.208, -0.0251], (1.182, 2.68], (-3.205, -1.208], (-1.208, -0.0251], (-0.0251, 1.182]]
Length: 1000
Categories (4, interval[float64]): [(-3.205, -1.208] < (-1.208, -0.0251] < (-0.0251, 1.182] < (1.182, 2.68]]

In [39]:
pd.value_counts(cats)

(-0.0251, 1.182]     400
(-1.208, -0.0251]    400
(1.182, 2.68]        100
(-3.205, -1.208]     100
dtype: int64

### Outliers

In [40]:
# Mean 0 and std 1.
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.031630,-0.064133,-0.025127,-0.009434
std,1.017965,0.978386,1.024862,0.994136
min,-3.372347,-2.862382,-3.112645,-2.829468
25%,-0.696864,-0.736802,-0.686469,-0.674919
50%,-0.054498,-0.067388,0.025861,0.009481
75%,0.702364,0.582211,0.683324,0.637848
max,3.454845,2.999190,3.609161,2.793166


In [41]:
data.hist(bins=30)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x110ece630>,
      dtype=object)

In [42]:
# Find values in one of the columns exceeding 2.7 in abs value.
# Extract data in the pandas way.
col = data[2]
col[np.abs(col) > 2.7]

135   -3.112645
293    3.609161
366   -2.707230
398   -2.715141
575    2.947099
615   -2.721390
700   -2.795615
953   -2.831778
Name: 2, dtype: float64

In [43]:
# Use pandas data inside numpy functions.
(np.abs(data) > 2.9).head()

,0,1,2,3
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False


In [44]:
# Find rows with at least one value larger than 2.9 in abs value.
data[(np.abs(data) > 2.9).any(axis=1)]

,0,1,2,3
5,-3.317669,0.895280,1.199629,0.381605
135,0.848669,0.060959,-3.112645,-0.217013
293,0.631801,-0.805615,3.609161,-0.219142
575,-1.940382,-0.269837,2.947099,0.324528
733,3.454845,-0.020427,0.984026,-1.486462
861,1.647859,2.999190,0.683966,-1.877642
932,2.925324,-0.090077,1.734451,0.006518
974,-3.372347,0.420341,1.010951,0.459729


### Permutation

### Indicator / Dummy variables

- Convert a categorical variable into a dummy (or indicator) variable
- If a column has k distinct values, then k columns with one-hot encoding will be added

#### pd.get_dummies()

In [45]:
df = pd.DataFrame({
    'key': ['b', 'b', 'a', 'c', 'a', 'b'],
    'data1': list(range(6))
})

df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [46]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [47]:
# One can add a prefix with "prefix".
pd.get_dummies(df['key'], prefix="key")

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [48]:
# One can combine pd.cut() to discretize with a pd.get_dummies() to compute indicator vars.

# Compute 10 random numbers.
np.random.seed(12345)
values = np.random.rand(10)

print("values=", values)

# Discretize with the given bins.
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

values= [0.92961609 0.31637555 0.18391881 0.20456028 0.56772503 0.5955447
 0.96451452 0.6531771  0.74890664 0.65356987]


,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String manipulation

### String methods

- split()
- join()

In [49]:
string = "hello my name is guido"
'guido' in string

True

In [50]:
# Find the index of a substring.
print(string.index('guido'))

print(string.find('guido'))

17
17


In [51]:
# Raises "ValueError: substring not found"
#string.index('Guido')

string.find('Guido')

-1

In [52]:
# Strip removes spaces on both sides.
' Guido    '.strip()

'Guido'

### Regex

In [53]:
import re

text = "foo bar\t baz \tqux"
# Split using 1 or more spaces as separator.
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [54]:
# If you want to call a regex several times, re.compile() speeds up things.
regex = re.compile('\s+')
regex.split(text)
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [55]:
# re.findall() finds all the matching substrings.
print(re.findall('\s', text))

[' ', '\t', ' ', ' ', '\t']


In [56]:
# re.finditer() returns an iterator to the regex matches.
print(re.finditer('\s', text))
print(list(re.finditer('\s', text)))

[<re.Match object; span=(3, 4), match=' '>, <re.Match object; span=(7, 8), match='\t'>, <re.Match object; span=(8, 9), match=' '>, <re.Match object; span=(12, 13), match=' '>, <re.Match object; span=(13, 14), match='\t'>]


In [57]:
# re.search() finds only the first.
m = re.search('\s', text)
m

<re.Match object; span=(3, 4), match=' '>

In [58]:
print(m.start(), m.end())

3 4


In [59]:
# Extract the match.
text[m.start():m.end()]

' '

In [60]:
# re.match() finds only the first, starting from the beginning of the string.

In [61]:
# Email pattern match.
# - alphanumerical chars . _ % + -
# - alphanumerical interspersed with .
# - capture the groups
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

regex = re.compile(pattern, flags=re.IGNORECASE)
m = regex.match('saggese@gmail.com')

m.groups()

('saggese', 'gmail', 'com')

In [62]:
text = """Dave dave@google.com
    Steve steve@gmail.com
    Rob rob@gmail.com
    Ryan ryan@yahoo.com
    """

# One can look for the patterns multiple times in the string.
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

### Vectorized string functions in pandas

In [63]:
data = {
    'Dave': 'dave@google.com',
    'Steve': 'steve@gmail.com',
    'Rob': 'rob@gmail.com',
    'Wes': np.nan
}
data = pd.Series(data)

data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [64]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [65]:
# You can use map() and string manipulation. The problem is that nan create problems,
# so one needs to handle it.
data.map(lambda x: x.lower() if isinstance(x, str) else "")

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                     
dtype: object

In [66]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [67]:
# Lots of string function have a wrapper.
data.str.__dir__()

['_inferred_dtype',
 '_is_categorical',
 '_parent',
 '_orig',
 '__frozen',
 '__module__',
 '__doc__',
 '__init__',
 '_validate',
 '__getitem__',
 '__iter__',
 '_wrap_result',
 '_get_series_list',
 'cat',
 'split',
 'rsplit',
 'partition',
 'rpartition',
 'get',
 'join',
 'contains',
 'match',
 'replace',
 'repeat',
 'pad',
 'center',
 'ljust',
 'rjust',
 'zfill',
 'slice',
 'slice_replace',
 'decode',
 'encode',
 'strip',
 'lstrip',
 'rstrip',
 'wrap',
 'get_dummies',
 'translate',
 'count',
 'startswith',
 'endswith',
 'findall',
 'extract',
 'extractall',
 'find',
 'rfind',
 'normalize',
 'index',
 'rindex',
 'len',
 '_doc_args',
 'lower',
 'upper',
 'title',
 'capitalize',
 'swapcase',
 'casefold',
 'isalnum',
 'isalpha',
 'isdigit',
 'isspace',
 'islower',
 'isupper',
 'istitle',
 'isnumeric',
 'isdecimal',
 '_make_accessor',
 '_freeze',
 '__setattr__',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__